In [69]:
import cv2
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import shutil

from tqdm import tqdm

In [85]:
epochs = 50
num_keypoint = 1 * 2
batch_size = 32
learning_rate = 1e-4

In [74]:
train = pd.read_csv("./train.csv")
valid = pd.read_csv("./valid.csv")
test = pd.read_csv("./test.csv")

print(f"train : {train.shape}")
print(f"valid : {valid.shape}")
print(f"test : {test.shape}")

train : (991, 3)
valid : (125, 3)
test : (125, 3)


In [75]:
root_dir = "./data"

# os.makedirs(root_dir+"/train")
# os.makedirs(root_dir+"/valid")
# os.makedirs(root_dir+'/test')

src = 'processed_images'
all_filename = os.listdir(src)

train_filename = [src + '/' + x for x in train['file']]
valid_filename = [src + '/' + x for x in valid['file']]
test_filename = [src + '/' + x for x in test['file']]

In [76]:
for name in tqdm(train_filename):
    shutil.copy(name, "./data/train")

100%|██████████████████████████████████████████████████████████████████████████████| 991/991 [00:00<00:00, 2103.56it/s]


In [77]:
for name in tqdm(valid_filename):
    shutil.copy(name, "./data/valid")

100%|██████████████████████████████████████████████████████████████████████████████| 125/125 [00:00<00:00, 2048.73it/s]


In [78]:
for name in tqdm(test_filename):
    shutil.copy(name, "./data/test")

100%|██████████████████████████████████████████████████████████████████████████████| 125/125 [00:00<00:00, 1865.27it/s]


In [79]:
# Augmentation Parameters
pixel_shifts = [8]

In [80]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D, ZeroPadding2D, GlobalAveragePooling2D
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from keras_preprocessing.image import ImageDataGenerator

In [88]:
datagen_train = ImageDataGenerator(rescale=1./255.,
                                   rotation_range=10,
                                   width_shift_range=0.1, 
                                   height_shift_range=0.1)

datagen_valid = ImageDataGenerator(rescale=1./255)

train_generator = datagen_train.flow_from_dataframe(dataframe=train,
                                                    directory="./data/train",
                                                    x_col="file",
                                                    y_col=['x', 'y'],
                                                    batch_size=batch_size,
                                                    seed=42,
                                                    shuffle=True,
                                                    class_mode=None,
                                                    target_size=(224, 224));



valid_generator=datagen_valid.flow_from_dataframe(dataframe=valid,
                                                  directory="./data/valid",
                                                  x_col="file",
                                                  y_col=['x', 'y'],
                                                  batch_size=batch_size,
                                                  seed=42,
                                                  shuffle=False,
                                                  class_mode=None,
                                                  target_size=(224, 224));

Found 991 validated image filenames.
Found 125 validated image filenames.


In [89]:
earlystop = EarlyStopping(patience=7)
model_checkpoint = ModelCheckpoint(filepath="./checkpoint/test", monitor="val_loss", save_best_only=True)

callbacks = [earlystop, model_checkpoint]

In [93]:
def get_model():
    # Load the pre-trained weights of MobileNetV2 and freeze the weights
    backbone = keras.applications.MobileNetV2(
        weights="imagenet", include_top=False, input_shape=(224, 224, 3)
    )
    backbone.trainable = False

    inputs = layers.Input((224, 224, 3))
    x = keras.applications.mobilenet_v2.preprocess_input(inputs)
    x = backbone(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(2)(x)

    return keras.Model(inputs, outputs, name="keypoint_detector")

In [94]:
model = get_model()

model.summary()

Model: "keypoint_detector"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.math.truediv_5 (TFOpLambd (None, 224, 224, 3)       0         
_________________________________________________________________
tf.math.subtract_5 (TFOpLamb (None, 224, 224, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
dropout_6 (Dropout)          (None, 7, 7, 1280)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 62720)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)         

In [97]:
model.compile(loss="mse", optimizer=keras.optimizers.Adam(learning_rate))

history = model.fit(train_generator,\
                    validation_data = valid_generator, \
                    epochs=epochs, \
                    verbose=1,\
                    callbacks = callbacks)

Epoch 1/50


ValueError: in user code:

    C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Administrator\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Administrator\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Administrator\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\traini